In [ ]:
import os
import time
import numpy as np
import pandas as pd
from nmf_methods.nmf_son.new import new as nmf_son_new
from nmf_methods.nmf_son.utils import save_results

np.random.seed(42)
np.set_printoptions(precision=3)

In [ ]:
RUN = True

EARLY_STOP = True
VERBOSE = False
SCALE_REG = True

In [ ]:
max_iters = 10000

In [ ]:
jasper_full = np.load('../../experimental/datasets/jasper_full.npz')['X']
jasper_3d = jasper_full.reshape(-1, 100, 100, order='F')

In [ ]:
ini_filepath = '../../experimental/saved_models/multi_size/r{}_ini.npz'
save_filepath = '../../experimental/saved_models/multi_size/r{}_l{}_mit{}.npz'
lambda_vals = [0.0001, 0.01, 0.1, 1, 10, 100, 1000]

data_2d = []

In [ ]:
m, _ = jasper_full.shape
dims = [3, 5, 7, 10, 12, 15, 20]
for dim in dims:
    M_3d = jasper_3d[:, :dim, :dim]
    M = M_3d.reshape(m, -1, order='F')
    m, n = M.shape
    r = n
    if os.path.exists(ini_filepath.format(r)):
        data = np.load(ini_filepath.format(r))
        ini_W = data['ini_W']
        ini_H = data['ini_H']
    else:
        ini_W = np.random.rand(m, r)
        ini_H = np.random.rand(r, n)
        with open(ini_filepath.format(r), 'wb') as fout:
            np.savez_compressed(fout, ini_W=ini_W, ini_H=ini_H)

    for _lam in lambda_vals:
        start_time = time.time()
        W, H, fscores, gscores, lvals = nmf_son_new(M, ini_W.copy(), ini_H.copy(), lam=_lam, itermax=max_iters, early_stop=EARLY_STOP, verbose=VERBOSE, scale_reg=SCALE_REG)
        time_taken = time.time() - start_time
        save_results(save_filepath.format(r, _lam, max_iters), W, H, fscores, gscores, lvals)
        data_2d.append([n, r, _lam, time_taken, fscores[-1], gscores[-1], lvals[-2]])

In [ ]:
df = pd.DataFrame(data_2d, columns=['n', 'rank', 'lambda', 'time_taken', 'fscore', 'gscore', 'scaled_lambda'])
df

In [ ]:
df.to_csv('../../experimental/saved_models/multi_size/results.csv')